In [6]:
#header
from datetime import date
from datetime import timedelta
import sys
import time
import psycopg2
import pandas as pd
import boto3
import mysql.connector
from mysql.connector import Error


#dates
today = date.today()+timedelta(days=1)
end_date=today.replace(day=1)-timedelta(days=1)
start_date=end_date.replace(day=1)

#functions 
def comma_separator(df, col):
    df['adgroup_id']=df['adgroup_id'].fillna('0')
    df['adgroup_id']=df['adgroup_id'].astype(int)
    constraints=df[col].apply(lambda x: str(x).split(',')).tolist()
    df_new=pd.DataFrame(constraints, index=df['adgroup_id'])
    df_new=df_new.apply(pd.Series).stack()
    df_new=pd.DataFrame(df_new)
    df_new.reset_index(inplace=True)
    df_new=df_new[['adgroup_id',0]]
    df_new.columns=['adgroup_id','constraints'] 
    df_new=df_new[~(df_new['constraints']=='None')]
    return df_new


def comma_separator_middle_extractor(df,col):
    df_new=comma_separator(df,col) 
    
    df_new['n1']=df_new['constraints'].apply(lambda x: x.split('_',1))
    df_new['final_constraints']=df_new['n1'].apply(lambda x: x[1].split('_')[0])
    df_new.drop(columns=['n1','constraints'],inplace=True)
    df_new.columns=['adgroup_id','constraints']
    
    return df_new
    

#connections

#Redshift connection 
red_con = psycopg2.connect(user="suraajhasija",password="x@DyDM79bX",host="redshift.xad.com",port="5439",
                              database="dwenigma")
redshift = red_con.cursor()

#xadcms connection

mysql_con= mysql.connector.connect(host='etl-int.db.xad.com',port='3336',user='analytics',password='analyticsxaddb',
                                    database='xadcms')            
xadcms = mysql_con.cursor()

#Postgres connection
post_con = psycopg2.connect(user="suraaj",password="b@bfgh9S8",host="poi-db.mgmt.xad.com",port="5432",
                              database="xaddb")
postgres= post_con.cursor()

########



#fetching the adgroup revenue and country
########################################
adgroups_rev = f"""
select a.*,c.revenue from     
(select campaign_id, adgroup_id,  market from dimensions.adgroup_dimension
where ((start_date between '{start_date}' and '{end_date}') or (end_date between '{start_date}' and '{end_date}'))
group by 1,2,3)a

inner join

(select * from (
select campaign_id, adgroup_id, count(distinct ad_date) as total_days, round(sum(revenue)*1.0,2) as revenue from (
select campaign_id, adgroup_id, ad_date, revenue*ex.exch_rate*1.0 as revenue from (
    select cds.*,cam.currency from (
                    select campaign_id, adgroup_id, date("timestamp") as ad_date, (xad_gross_revenue) as revenue
                    from campaign_daily_summary where date("timestamp") between '{start_date}' and '{end_date}' 
                    group by 1,2,3,4 ) cds
              inner join campaign_dimension cam
              on cds.campaign_id=cam.campaign_id
              group by 1,2,3,4,5
                                    )cam_cds
left join exchange_rate_dimension ex 
on (cam_cds.currency=ex.source_currency and cam_cds.ad_date=ex."date")
where ex.target_currency = 'USD'
group by 1,2,3,4
)rev group by 1,2 having sum(revenue)*1.0 >0
)x)c
on ( a.campaign_id=c.campaign_id and a.adgroup_id=c.adgroup_id)
"""
#executing queries
redshift.execute(adgroups_rev)
adgroup_records = redshift.fetchall()



df_adgroups=pd.DataFrame(adgroup_records, columns=['campaign_id','adgroup_id','market','revenue'])
    
adgroup=set(df_adgroups['adgroup_id'].values)
adgroups=str(adgroup)[1:-1]        
########################################

#fetching tags and brands for tracking 


tracking = """
select adgroup_id,campaign_id,a.mobileSite_id, m.dst_tags, m.dst_brands from
(select id as adgroup_id,campaign_id,mobileSite_id from adgroup where del = 0 and id in ({})) as a inner join mobilesite m on a.mobileSite_id = m.id 
where (m.dst_tags is not null or m.dst_brands is not null ) and (m.dst_tags not in ('') or m.dst_tags not in (''))group by 1,2,3,4,5
""".format(adgroups)

xadcms.execute(tracking)
tracking = xadcms.fetchall() 


df_tracked=pd.DataFrame(tracking, columns=['adgroup_id','campaign_id','mobilesite_id','tags','brands'])
df_tracked_brands=comma_separator(df_tracked[['adgroup_id','brands']],'brands')
df_tracked_tags=comma_separator(df_tracked[['adgroup_id','tags']],'tags')

########################################
########################################

#Fetching targeting adgroups

targeting = """
    select adGroup_id,targetType, constraints from targetprofile where targetType in (14,23,28,45,56,24,15) and 
    adgroup_id in ({}) group by 1,2,3""".format(adgroups)

xadcms.execute(targeting)
targeting = xadcms.fetchall()


df_targeted=pd.DataFrame(targeting, columns=['adgroup_id','target_type','constraints'])

#Footprints brands TargetType=14
df_foot_brands=comma_separator_middle_extractor(df_targeted[df_targeted['target_type']==14][['adgroup_id','constraints']],'constraints')

#Location Audience brands TargetType=23
df_LocAud_brands=comma_separator(df_targeted[df_targeted['target_type']==23][['adgroup_id','constraints']],'constraints')

# BA brands TargetType=28
df_ba_brands=comma_separator(df_targeted[df_targeted['target_type']==28][['adgroup_id','constraints']],'constraints')

#Raw file for BA segments 

ba= pd.read_excel('/Users/suraajhasija/Desktop/Blueprints Revenue/BASegments.xlsx', sheet_name='Basic Rules')
ba_brands=ba[ba['Feature']=='fpBrand'][['Segment Code','Feature Value']]
ba_category=ba[ba['Feature']=='fpSIC'][['Segment Code','Feature Value']]

#joining with the static file to get the brands against segment codes
df_ba_brands_final=df_ba_brands.merge(ba_brands, left_on='constraints', right_on='Segment Code', how='inner')

#some segment codes were blank (NaN) after this -- check this 
#some renaming efforts
df_ba_brands_final=df_ba_brands_final[['adgroup_id','Feature Value']]
df_ba_brands_final.columns=['adgroup_id','constraints']
df_ba_brands_final.dropna(subset=['constraints'],inplace=True) 

### Final table for Targeted brands
df_targeted_brands=pd.concat([df_ba_brands_final,df_LocAud_brands,df_foot_brands]).drop_duplicates()  #final


#LocationGroup Tags TargetType=45
df_LG_tags=comma_separator_middle_extractor(df_targeted[df_targeted['target_type']==45][['adgroup_id','constraints']],'constraints')

#LOCATIONAUDIENCE_LOCATIONGROUPS Tags TargetType=56
df_LA_LG_tags=comma_separator(df_targeted[df_targeted['target_type']==56][['adgroup_id','constraints']],'constraints')

#Final table for Targeted Tags
df_targeted_tags=pd.concat([df_LA_LG_tags,df_LG_tags]).drop_duplicates()    #final

#Category
#LOCATION AUDIENCE SIC CODE TargetType=24

df_LA_cat=comma_separator(df_targeted[df_targeted['target_type']==24][['adgroup_id','constraints']],'constraints')

#footprints category TargetType=15
df_foot_cat=comma_separator_middle_extractor(df_targeted[df_targeted['target_type']==15][['adgroup_id','constraints']],'constraints')

# BA Category
df_ba_category=comma_separator(df_targeted[df_targeted['target_type']==28][['adgroup_id','constraints']],'constraints')

#joining with the static file to get the brands against segment codes
df_ba_category_final=df_ba_category.merge(ba_category, left_on='constraints', right_on='Segment Code', how='inner')

#some segment codes were blank (NaN) after this -- check this 
#some renaming efforts
df_ba_category_final=df_ba_category_final[['adgroup_id','Feature Value']]
df_ba_category_final.columns=['adgroup_id','constraints']
df_ba_category_final.dropna(subset=['constraints'],inplace=True) 

### Final table for Targeted Category
df_targeted_category=pd.concat([df_foot_cat,df_LA_cat,df_ba_category_final]).drop_duplicates()  #final


########################################
########################################



df_tags=pd.concat([df_tracked_tags, df_targeted_tags]).drop_duplicates() 

df_brands=pd.concat([df_tracked_brands, df_targeted_brands]).drop_duplicates() 

#building final tags, category & brands tables with their constraints
df_tags_country=df_tags.merge(df_adgroups, on='adgroup_id', how='inner')
df_tags_country=df_tags_country[['adgroup_id','market','constraints']]

df_brands_country=df_brands.merge(df_adgroups, on='adgroup_id', how='inner')
df_brands_country=df_brands_country[['adgroup_id','market','constraints']]

df_category_country=df_targeted_category.merge(df_adgroups, on='adgroup_id', how='inner')
df_category_country=df_category_country[['adgroup_id','market','constraints']]

print(df_category_country.shape)
print(df_brands_country.shape)
print(df_tags_country.shape)


#fetching POIs of tags from POI DB

df_tags_pois=pd.DataFrame()
countries_tags=list(df_brands_country.market.value_counts().index)
#countries_tags.remove('US') #add US after verification

for i in countries_tags:
    country_tags_poi=f""" select a.tag, b.id as poi_id, '{i}' as country
        from pgadmin.business_tag a 
        inner join pgadmin.poi b on a.id=b.id 
        where b.del=false and a.tag in ({str(set(map(int,df_tags_country[df_tags_country.market==i]['constraints'].values)))[1:-1]}) 
        and b.flagged in (0,6)
        and country in ('{i}')
        group by 1,2,3"""
    postgres.execute(country_tags_poi)
    df_t=pd.DataFrame(postgres.fetchall(),columns=['tags','poi','country'])
    df_tags_pois = pd.concat([df_tags_pois,df_t], ignore_index = True).drop_duplicates().reset_index(drop=True)






#### final tag & Pois
df_tags_country.constraints=df_tags_country.constraints.astype(int)
df_tags_final=df_tags_country.merge(df_tags_pois, left_on=['constraints','market'], right_on=['tags','country'], how='inner') 

print("df_tags_final.shape",df_tags_final.shape)


#fetching pois of brands from POI DB

df_brands_pois=pd.DataFrame()

countries_brands=list(df_brands_country.market.value_counts().index)
#countries_brands.remove('US')

for i in countries_brands:
    country_brands_poi=f""" select b."brandId", b.id as poi_id, '{i}' as country
        from pgadmin.poi b 
        where b.del=false and b."brandId" in ({str(set(map(int,df_brands_country[df_brands_country.market==i]['constraints'].values)))[1:-1]}) 
        and b.flagged in (0,6)
        and country in ('{i}')
        group by 1,2,3"""
    postgres.execute(country_brands_poi)
    df_b=pd.DataFrame(postgres.fetchall(),columns=['brands','poi','country'])
    df_brands_pois = pd.concat([df_brands_pois,df_b], ignore_index = True).drop_duplicates().reset_index(drop=True)




df_brands_country.constraints=df_brands_country.constraints.astype(int)

#### final brand & Pois
df_brands_final=df_brands_country.merge(df_brands_pois, left_on=['constraints','market'], right_on=['brands','country'], how='inner')
print("df_brands_final.shape",df_brands_final.shape)

#fetching pois of category from POI DB
post_con = psycopg2.connect(user="suraaj",password="b@bfgh9S8",host="poi-db.mgmt.xad.com",port="5432",
                              database="xaddb")
postgres= post_con.cursor()

df_category_pois=pd.DataFrame()
countries_category=list(df_category_country.market.value_counts().index) 
#countries_category.remove('US')

for i in countries_category:
    country_category_poi=f""" select b."primaryCategory", b.id as poi_id, '{i}' as country
        from pgadmin.poi b 
        where b.del=false and cast (b."primaryCategory" as int) in ({str(set(map(int,df_category_country[df_category_country.market==i]['constraints'].values)))[1:-1]}) 
        and b.flagged in (0,6)
        and country in ('{i}')
        group by 1,2,3"""
    postgres.execute(country_category_poi)
    df_c=pd.DataFrame(postgres.fetchall(),columns=['category','poi','country'])
    df_category_pois = pd.concat([df_category_pois,df_c], ignore_index = True).drop_duplicates().reset_index(drop=True)

df_category_country.constraints=df_category_country.constraints.astype(str)

#### final category & Pois
df_category_final=df_category_country.merge(df_category_pois, left_on=['constraints','market'], right_on=['category','country'], how='inner')

print("df_category_final.shape",df_category_final.shape)

#union and removinf duplicates
df_adgroups_pois=pd.concat([df_tags_final[['adgroup_id','poi']], df_brands_final[['adgroup_id','poi']],df_category_final[['adgroup_id','poi']]]).drop_duplicates()

#counting pois in each adgroup (QC REQUIRED HERE FOR REVENUE) IMP IMP IMP
adgroup_count_pois=df_adgroups_pois.groupby(['adgroup_id'],as_index=False).count()
print("adgroup_count_pois")

#bringing revenue for each each adgroup 
adgroup_count_poi_rev=adgroup_count_pois.merge(df_adgroups, on='adgroup_id', how='inner')
print("adgroup_count_poi_rev")
#calculating per poi revenue 
adgroup_count_poi_rev.revenue=adgroup_count_poi_rev.revenue.astype(int)
adgroup_count_poi_rev['per_poi_revenue']=adgroup_count_poi_rev['revenue']/adgroup_count_poi_rev['poi']

#joining per POI revenue with main table
df_final=df_adgroups_pois.merge(adgroup_count_poi_rev, on ='adgroup_id', how='inner')
print("df_final")
df_final=df_final[['adgroup_id','poi_x','per_poi_revenue']]

#aggregating at POI level
# df_final.poi_x=df_final.poi_x.astype(str)
df_poi_revenue=df_final.groupby(['poi_x'],as_index=False).sum()
df_poi_revenue.drop(columns='adgroup_id',inplace=True)

<ipython-input-6-d1ce990cf7b7>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['adgroup_id']=df['adgroup_id'].fillna('0')
<ipython-input-6-d1ce990cf7b7>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['adgroup_id']=df['adgroup_id'].astype(int)


(39105, 3)
(471527, 3)
(5219, 3)
df_tags_final.shape (2486617, 6)
df_brands_final.shape (140188517, 6)
df_category_final.shape (359081360, 6)


In [12]:
df_poi_revenue

,poi_x,per_poi_revenue
0,81,0.000099
1,108,0.000576
2,118,0.079799
3,123,0.001029
4,160,0.003727
...,...,...
3890804,103714718,0.002022
3890805,103714719,0.002022
3890806,103714720,0.002022
3890807,103714721,0.003003


## QC

In [13]:
sum(df_poi_revenue['per_poi_revenue']) #final rev check

51595.99999981269

In [14]:
sum(adgroup_count_poi_rev['revenue']) #backtracking

51596

In [15]:
sum(df_final['per_poi_revenue'])

51595.999995764076

In [16]:
df_final['adgroup_id'].nunique()  #adgroups decreased

6596

In [17]:
set(adgroup_count_poi_rev['adgroup_id']).difference(set(df_final['adgroup_id']))

In [24]:
df_adgroups.adgroup_id.nunique() #oringinal number of adgroups. Need to find rationale behind it


8487

In [25]:
df_tracked.adgroup_id.nunique()

2967

In [26]:
df_targeted.adgroup_id.nunique()  

5525

In [29]:
len(set(df_targeted.adgroup_id).difference(set(df_tracked.adgroup_id)))  #overlapping

3747

#### check after running again

In [ ]:
#fetching brands, category corresponding to every POI

brand_category_fetch=f"""select id,"brandId", "primaryCategory" from pgadmin.poi where id in ({str(set(map(int,df_poi_revenue['poi_x'].values)))[1:-1]})"""
postgres.execute(brand_category_fetch)
df_poi_details=pd.DataFrame(postgres.fetchall(),columns=['poi','brand','category'])

#cleaning
df=df_poi_revenue.merge(df_poi_details,left_on='poi_x', right_on='poi', how='inner')
df.brand=df.brand.astype(str)
df.poi=df.poi.astype(str)
df.drop(columns='poi_x', inplace=True)


#creating final table
df_tops=df.groupby(['category','brand'],as_index=False).sum().sort_values(['per_poi_revenue'], ascending=False)

In [45]:
###################
df_tops
#################

,category,brand,per_poi_revenue
9991,999999,0,3339.920427
9344,821103,0,2105.086677
9362,822101,0,1696.042209
1127,481207,16521,1595.504742
2341,541103,935,1310.029800
...,...,...,...
7606,721201,3393,0.000000
7605,721201,3392,0.000000
7604,721201,3339,0.000000
7602,721201,3336,0.000000


In [50]:
######################
df_poi_revenue
######################

,poi_x,per_poi_revenue
0,81,0.000099
1,108,0.000576
2,118,0.079799
3,123,0.001029
4,160,0.003727
...,...,...
3890804,103714718,0.002022
3890805,103714719,0.002022
3890806,103714720,0.002022
3890807,103714721,0.003003
